In [2]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the NLLB model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

In [ ]:
# Read the contents of the file
with open("to_be_localized.txt", "r", encoding="utf-8") as file:
    contents = file.read()

# Split the contents by localization unit
localization_units = contents.split("--")

# Process each localization unit
for unit in localization_units:
    # Find the text enclosed in <Literal> tags
    match = re.search(r"<Literal>(.*?)<\/Literal>", unit, re.DOTALL)
    if match:
        english_text = match.group(1)
        
        # Tokenize the English text and translate to Spanish
        inputs = tokenizer(english_text, return_tensors="pt", src_lang="en_XX", tgt_lang="es_XX")
        translation_outputs = model.generate(**inputs)
        spanish_translation = tokenizer.decode(translation_outputs[0], skip_special_tokens=True)
        
        # Replace the English text with the Spanish translation in the localization unit
        translated_unit = unit.replace(english_text, spanish_translation)
        
        # Print or save the translated unit
        print(translated_unit)

# Note:
# The script assumes that the NLLB model supports Spanish translation.
# The model and tokenizer names ("facebook/nllb-200-distilled-600M") are placeholders and should be replaced with the correct names of the NLLB model and tokenizer you intend to use.
